<a href="https://colab.research.google.com/github/Ddhawan2003/FplAnalytics/blob/main/ScorePredictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
matches = pd.read_csv('matches_final.csv', index_col=0)

In [ ]:
matches.shape

(3040, 30)

In [ ]:
matches['team'].value_counts()

team
Manchester City             152
Manchester United           152
Everton                     152
Arsenal                     152
Brighton and Hove Albion    152
Crystal Palace              152
West Ham United             152
Wolverhampton Wanderers     152
Newcastle United            152
Tottenham Hotspur           152
Aston Villa                 152
Liverpool                   152
Chelsea                     152
Fulham                      114
Brentford                   114
Burnley                     114
Leicester City              114
Leeds United                114
Southampton                 114
Bournemouth                  76
Nottingham Forest            76
Sheffield United             76
Norwich City                 38
Luton Town                   38
Watford                      38
West Bromwich Albion         38
Name: count, dtype: int64

In [ ]:
matches["round"].value_counts()

,count
round,
Matchweek 1,80
Matchweek 30,80
Matchweek 23,80
Matchweek 24,80
Matchweek 25,80
Matchweek 18,80
Matchweek 26,80
Matchweek 27,80
Matchweek 28,80


In [ ]:
matches.dtypes

date             object
time             object
comp             object
round            object
day              object
venue            object
result           object
gf                int64
ga                int64
opponent         object
xg              float64
xga             float64
poss            float64
attendance      float64
captain          object
formation        object
referee          object
match report     object
notes           float64
sh              float64
sot             float64
dist            float64
fk              float64
pk                int64
pkatt             int64
season            int64
team             object
dtype: object

In [ ]:
del matches["comp"]
del matches["notes"]
matches["date"] = pd.to_datetime(matches["date"])

In [ ]:
matches["target"] = (matches["result"] == "W").astype("int")
matches.head()

,date,time,round,day,venue,result,gf,ga,opponent,xg,...,match report,sh,sot,dist,fk,pk,pkatt,season,team,target
1,2023-08-11,20:00,Matchweek 1,Fri,Away,W,3,0,Burnley,1.9,...,Match Report,17.0,8.0,13.9,0.0,0,0,2024,Manchester City,1
3,2023-08-19,20:00,Matchweek 2,Sat,Home,W,1,0,Newcastle Utd,1.0,...,Match Report,14.0,4.0,17.9,0.0,0,0,2024,Manchester City,1
4,2023-08-27,14:00,Matchweek 3,Sun,Away,W,2,1,Sheffield Utd,3.5,...,Match Report,29.0,9.0,17.3,2.0,0,1,2024,Manchester City,1
5,2023-09-02,15:00,Matchweek 4,Sat,Home,W,5,1,Fulham,2.2,...,Match Report,6.0,4.0,14.8,0.0,1,1,2024,Manchester City,1
6,2023-09-16,15:00,Matchweek 5,Sat,Away,W,3,1,West Ham,3.6,...,Match Report,29.0,13.0,16.4,1.0,0,0,2024,Manchester City,1


In [ ]:
matches["venue_code"] = matches["venue"].astype("category").cat.codes
matches["opp_code"] = matches["opponent"].astype("category").cat.codes
matches["hour"] = matches["time"].str.replace(":.+", "", regex=True).astype("int")
matches["day_code"] = matches["date"].dt.dayofweek

In [ ]:
matches.head()

,date,time,round,day,venue,result,gf,ga,opponent,xg,...,fk,pk,pkatt,season,team,target,venue_code,opp_code,hour,day_code
1,2023-08-11,20:00,Matchweek 1,Fri,Away,W,3,0,Burnley,1.9,...,0.0,0,0,2024,Manchester City,1,0,5,20,4
3,2023-08-19,20:00,Matchweek 2,Sat,Home,W,1,0,Newcastle Utd,1.0,...,0.0,0,0,2024,Manchester City,1,1,16,20,5
4,2023-08-27,14:00,Matchweek 3,Sun,Away,W,2,1,Sheffield Utd,3.5,...,2.0,0,1,2024,Manchester City,1,0,19,14,6
5,2023-09-02,15:00,Matchweek 4,Sat,Home,W,5,1,Fulham,2.2,...,0.0,1,1,2024,Manchester City,1,1,9,15,5
6,2023-09-16,15:00,Matchweek 5,Sat,Away,W,3,1,West Ham,3.6,...,1.0,0,0,2024,Manchester City,1,0,24,15,5


In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=5000, min_samples_split=10, random_state=2)
train = matches[matches["date"] < '2024-01-01']
test = matches[matches["date"] > '2024-01-01']
predictors = ["venue_code", "opp_code", "hour", "day_code"]
rf.fit(train[predictors], train["target"])

RandomForestClassifier(min_samples_split=10, n_estimators=5000, random_state=2)

In [ ]:
preds = rf.predict(test[predictors])
from sklearn.metrics import accuracy_score
error = accuracy_score(test["target"], preds)
error

0.6502732240437158

In [ ]:
# prompt: view predictions now for team_name tottenham

test["predicted"] = preds
combined = pd.concat([test[test["team"] == "Tottenham Hotspur"],
                       test[test["opponent"] == "Tottenham"]])
combined = combined.sort_values("date")
combined[["date", "team", "opponent", "result", "predicted"]]


<ipython-input-17-863063f311b8>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test["predicted"] = preds


,date,team,opponent,result,predicted
22,2024-01-14,Tottenham Hotspur,Manchester Utd,D,0
29,2024-01-14,Manchester United,Tottenham,D,0
24,2024-01-31,Tottenham Hotspur,Brentford,W,0
24,2024-01-31,Brentford,Tottenham,L,0
25,2024-02-03,Tottenham Hotspur,Everton,D,0
29,2024-02-03,Everton,Tottenham,D,1
26,2024-02-10,Tottenham Hotspur,Brighton,W,0
32,2024-02-10,Brighton and Hove Albion,Tottenham,L,0
27,2024-02-17,Tottenham Hotspur,Wolves,L,0
29,2024-02-17,Wolverhampton Wanderers,Tottenham,W,0


In [ ]:
combined = pd.DataFrame(dict(actual=test["target"], predicted=preds))
pd.crosstab(index=combined["actual"], columns=combined["predicted"])

predicted,0,1
actual,,
0,180,49
1,79,58


In [ ]:
from sklearn.metrics import precision_score

precision_score(test["target"], preds)

0.5420560747663551

In [ ]:
grouped_matches = matches.groupby("team")
group = grouped_matches.get_group("Manchester City").sort_values("date")
def rolling_averages(group, cols, new_cols):
    group = group.sort_values("date")
    rolling_stats = group[cols].rolling(3, closed='left').mean()
    group[new_cols] = rolling_stats
    group = group.dropna(subset=new_cols)
    return group
cols = ["gf", "ga", "sh", "sot", "dist", "fk", "pk", "pkatt"]
new_cols = [f"{c}_rolling" for c in cols]

rolling_averages(group, cols, new_cols)


,date,time,round,day,venue,result,gf,ga,opponent,xg,...,hour,day_code,gf_rolling,ga_rolling,sh_rolling,sot_rolling,dist_rolling,fk_rolling,pk_rolling,pkatt_rolling
5,2020-10-17,17:30,Matchweek 5,Sat,Home,W,1,0,Arsenal,1.3,...,17,5,2.000000,2.333333,17.333333,4.666667,19.700000,1.333333,0.333333,0.333333
7,2020-10-24,12:30,Matchweek 6,Sat,Away,D,1,1,West Ham,1.0,...,12,5,1.333333,2.000000,17.333333,3.666667,18.566667,0.666667,0.000000,0.000000
9,2020-10-31,12:30,Matchweek 7,Sat,Away,W,1,0,Sheffield Utd,1.6,...,12,5,1.000000,0.666667,16.666667,4.333333,18.933333,0.666667,0.000000,0.000000
11,2020-11-08,16:30,Matchweek 8,Sun,Home,D,1,1,Liverpool,1.4,...,16,6,1.000000,0.333333,14.333333,6.666667,19.033333,1.000000,0.000000,0.000000
12,2020-11-21,17:30,Matchweek 9,Sat,Away,L,0,2,Tottenham,1.4,...,17,5,1.000000,0.666667,12.000000,5.666667,20.000000,1.000000,0.000000,0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51,2024-04-28,16:30,Matchweek 35,Sun,Away,W,2,0,Nott'ham Forest,0.9,...,16,6,4.333333,1.000000,22.666667,8.666667,16.666667,0.333333,0.333333,0.333333
52,2024-05-04,17:30,Matchweek 36,Sat,Home,W,5,1,Wolves,3.7,...,17,5,3.666667,0.333333,20.333333,7.666667,17.533333,0.666667,0.333333,0.333333
53,2024-05-11,12:30,Matchweek 37,Sat,Away,W,4,0,Fulham,3.2,...,12,5,3.666667,0.333333,14.333333,7.000000,17.000000,0.666667,0.666667,0.666667
54,2024-05-14,20:00,Matchweek 34,Tue,Away,W,2,0,Tottenham,2.4,...,20,1,3.666667,0.333333,14.666667,7.666667,17.200000,0.333333,1.000000,1.000000


In [ ]:
matches_rolling = matches.groupby("team").apply(lambda x: rolling_averages(x, cols, new_cols))
matches_rolling = matches_rolling.droplevel('team')
matches_rolling.index = range(matches_rolling.shape[0])
matches_rolling

,date,time,round,day,venue,result,gf,ga,opponent,xg,...,hour,day_code,gf_rolling,ga_rolling,sh_rolling,sot_rolling,dist_rolling,fk_rolling,pk_rolling,pkatt_rolling
0,2020-10-04,14:00,Matchweek 4,Sun,Home,W,2,1,Sheffield Utd,0.6,...,14,6,2.000000,1.333333,8.000000,3.666667,14.633333,0.666667,0.000000,0.000000
1,2020-10-17,17:30,Matchweek 5,Sat,Away,L,0,1,Manchester City,0.9,...,17,5,1.666667,1.666667,5.666667,3.666667,15.366667,0.000000,0.000000,0.000000
2,2020-10-25,19:15,Matchweek 6,Sun,Home,L,0,1,Leicester City,0.7,...,19,6,1.000000,1.666667,7.000000,3.666667,16.566667,0.666667,0.000000,0.000000
3,2020-11-01,16:30,Matchweek 7,Sun,Away,W,1,0,Manchester Utd,1.0,...,16,6,0.666667,1.000000,9.666667,4.000000,16.566667,1.000000,0.000000,0.000000
4,2020-11-08,19:15,Matchweek 8,Sun,Home,L,0,3,Aston Villa,1.5,...,19,6,0.333333,0.666667,9.666667,2.666667,19.333333,1.000000,0.333333,0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2954,2024-04-24,19:45,Matchweek 29,Wed,Home,L,0,1,Bournemouth,0.5,...,19,2,1.000000,2.000000,9.666667,4.000000,20.533333,0.333333,0.333333,0.333333
2955,2024-04-27,15:00,Matchweek 35,Sat,Home,W,2,1,Luton Town,1.2,...,15,5,0.666667,1.666667,10.333333,3.333333,18.766667,0.000000,0.000000,0.000000
2956,2024-05-04,17:30,Matchweek 36,Sat,Away,L,1,5,Manchester City,0.3,...,17,5,0.666667,1.333333,11.000000,4.000000,19.666667,0.000000,0.000000,0.000000
2957,2024-05-11,15:00,Matchweek 37,Sat,Home,L,1,3,Crystal Palace,1.6,...,15,5,1.000000,2.333333,10.000000,3.333333,15.966667,0.000000,0.000000,0.000000


In [30]:
def make_predictions(data, predictors):
    train = data[data["date"] < '2024-01-01']
    test = data[data["date"] > '2024-01-01']
    rf.fit(train[predictors], train["target"])
    preds = rf.predict(test[predictors])
    combined = pd.DataFrame(dict(actual=test["target"], predicted=preds), index=test.index)
    error = precision_score(test["target"], preds)
    return combined, error

In [50]:
combined, error = make_predictions(matches_rolling, predictors + new_cols)

In [51]:
error

0.6

In [52]:
combined = combined.merge(matches_rolling[["date", "team", "opponent", "result"]], left_index=True, right_index=True)
combined.head(10)

,actual,predicted,date,team,opponent,result
131,1,0,2024-01-20,Arsenal,Crystal Palace,W
132,1,1,2024-01-30,Arsenal,Nott'ham Forest,W
133,1,0,2024-02-04,Arsenal,Liverpool,W
134,1,1,2024-02-11,Arsenal,West Ham,W
135,1,1,2024-02-17,Arsenal,Burnley,W
136,1,1,2024-02-24,Arsenal,Newcastle Utd,W
137,1,1,2024-03-04,Arsenal,Sheffield Utd,W
138,1,1,2024-03-09,Arsenal,Brentford,W
139,0,0,2024-03-31,Arsenal,Manchester City,D
140,1,0,2024-04-03,Arsenal,Luton Town,W


In [44]:
# prompt: show different opponent team names and different team names

print(matches["opponent"].unique())
print(matches["team"].unique())


['Burnley' 'Newcastle Utd' 'Sheffield Utd' 'Fulham' 'West Ham'
 "Nott'ham Forest" 'Wolves' 'Arsenal' 'Brighton' 'Manchester Utd'
 'Bournemouth' 'Chelsea' 'Liverpool' 'Tottenham' 'Aston Villa'
 'Luton Town' 'Crystal Palace' 'Everton' 'Brentford' 'Manchester City'
 'Southampton' 'Leicester City' 'Leeds United' 'Norwich City' 'Watford'
 'West Brom']
['Manchester City' 'Arsenal' 'Liverpool' 'Aston Villa' 'Tottenham Hotspur'
 'Chelsea' 'Newcastle United' 'Manchester United' 'West Ham United'
 'Crystal Palace' 'Brighton and Hove Albion' 'Bournemouth' 'Fulham'
 'Wolverhampton Wanderers' 'Everton' 'Brentford' 'Nottingham Forest'
 'Luton Town' 'Burnley' 'Sheffield United' 'Leicester City' 'Leeds United'
 'Southampton' 'Watford' 'Norwich City' 'West Bromwich Albion']


In [53]:
class MissingDict(dict):
    __missing__ = lambda self, key: key

map_values = {
    "Brighton and Hove Albion": "Brighton",
    "Manchester United": "Manchester Utd",
    "Newcastle United": "Newcastle Utd",
    "Tottenham Hotspur": "Tottenham",
    "West Ham United": "West Ham",
    "Wolverhampton Wanderers": "Wolves",
    "Leicester City": "Leicester City",
    "Leeds United": "Leeds United",
    "Southampton": "Southampton",
    "Watford": "Watford",
    "Norwich City": "Norwich City",
    "West Bromwich Albion": "West Brom",
    "Burnley": "Burnley",
    "Sheffield United": "Sheffield Utd",
    "Fulham": "Fulham",
    "Nottingham Forest": "Nott'ham Forest",
    "Arsenal": "Arsenal",
    "Manchester City": "Manchester City",
    "Bournemouth": "Bournemouth",
    "Chelsea": "Chelsea",
    "Liverpool": "Liverpool",
    "Aston Villa": "Aston Villa",
    "Luton Town": "Luton Town",
    "Crystal Palace": "Crystal Palace",
    "Everton": "Everton",
    "Brentford": "Brentford"
}
mapping = MissingDict(**map_values)
combined["new_team"] = combined["team"].map(mapping)
merged = combined.merge(combined, left_on=["date", "new_team"], right_on=["date", "opponent"])
merged

,actual_x,predicted_x,date,team_x,opponent_x,result_x,new_team_x,actual_y,predicted_y,team_y,opponent_y,result_y,new_team_y
0,1,0,2024-01-20,Arsenal,Crystal Palace,W,Arsenal,0,0,Crystal Palace,Arsenal,L,Crystal Palace
1,1,1,2024-01-30,Arsenal,Nott'ham Forest,W,Arsenal,0,0,Nottingham Forest,Arsenal,L,Nott'ham Forest
2,1,0,2024-02-04,Arsenal,Liverpool,W,Arsenal,0,1,Liverpool,Arsenal,L,Liverpool
3,1,1,2024-02-11,Arsenal,West Ham,W,Arsenal,0,0,West Ham United,Arsenal,L,West Ham
4,1,1,2024-02-17,Arsenal,Burnley,W,Arsenal,0,0,Burnley,Arsenal,L,Burnley
...,...,...,...,...,...,...,...,...,...,...,...,...,...
361,0,0,2024-04-24,Wolverhampton Wanderers,Bournemouth,L,Wolves,1,0,Bournemouth,Wolves,W,Bournemouth
362,1,0,2024-04-27,Wolverhampton Wanderers,Luton Town,W,Wolves,0,0,Luton Town,Wolves,L,Luton Town
363,0,0,2024-05-04,Wolverhampton Wanderers,Manchester City,L,Wolves,1,1,Manchester City,Wolves,W,Manchester City
364,0,0,2024-05-11,Wolverhampton Wanderers,Crystal Palace,L,Wolves,1,1,Crystal Palace,Wolves,W,Crystal Palace


In [54]:
merged[(merged["predicted_x"] == 1) & (merged["predicted_y"] ==0)]["actual_x"].value_counts()

actual_x
1    56
0    36
Name: count, dtype: int64

In [55]:
# prompt: analyze how good this merged table is at predicting the outcome of a match

# Calculate the accuracy of predictions
total_predictions = merged.shape[0]
correct_predictions = merged[(merged["predicted_x"] == merged["actual_x"]) & (merged["predicted_y"] == merged["actual_y"])].shape[0]
accuracy = correct_predictions / total_predictions

print(f"Accuracy of the merged table: {accuracy:.2f}")

# Analyze the precision of predictions for each team
team_precisions = {}
for team in merged["new_team"].unique():
  team_data = merged[merged["new_team"] == team]
  true_positives = team_data[(team_data["predicted_x"] == 1) & (team_data["actual_x"] == 1)].shape[0]
  predicted_positives = team_data[team_data["predicted_x"] == 1].shape[0]
  if predicted_positives > 0:
    precision = true_positives / predicted_positives
    team_precisions[team] = precision

print("\nPrecision for each team:")
for team, precision in team_precisions.items():
  print(f"{team}: {precision:.2f}")

# Analyze the recall of predictions for each team
team_recalls = {}
for team in merged["new_team"].unique():
  team_data = merged[merged["new_team"] == team]
  true_positives = team_data[(team_data["predicted_x"] == 1) & (team_data["actual_x"] == 1)].shape[0]
  actual_positives = team_data[team_data["actual_x"] == 1].shape[0]
  if actual_positives > 0:
    recall = true_positives / actual_positives
    team_recalls[team] = recall

print("\nRecall for each team:")
for team, recall in team_recalls.items():
  print(f"{team}: {recall:.2f}")

# Analyze the distribution of predicted outcomes
predicted_wins = merged[(merged["predicted_x"] == 1) & (merged["predicted_y"] == 0)].shape[0]
predicted_losses = merged[(merged["predicted_x"] == 0) & (merged["predicted_y"] == 1)].shape[0]
predicted_draws = merged[(merged["predicted_x"] == merged["predicted_y"])].shape[0]

print("\nDistribution of predicted outcomes:")
print(f"Predicted wins: {predicted_wins}")
print(f"Predicted losses: {predicted_losses}")
print(f"Predicted draws: {predicted_draws}")


Accuracy of the merged table: 0.44


KeyError: 'new_team'